In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader

In [2]:
dtype = torch.cuda.FloatTensor # Uncomment this to run on GPU

In [3]:
sortedFeatureCorr = pd.read_csv("dataset/corr.csv")
data = pd.read_csv("dataset/preprocessed.csv")

In [4]:
# time stamp will add

In [5]:
first_feature_count =           22
last_feature_count =            21

In [6]:
a = sortedFeatureCorr.iloc[:, 0].values

In [7]:
print(a.shape)

(43,)


In [8]:
for i in range(first_feature_count , sortedFeatureCorr.shape[0] - last_feature_count):
    data = data.drop(a[i], axis=1)
data=data.drop(list(data.columns)[0], axis=1)


In [9]:
data_npWNan = data.iloc[:, :].values

data_np = data_npWNan[data_npWNan[:,-1]!= -1 ]
unique_elements, counts_elements = np.unique(data_np[:,-1], return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[     1      2      3      4      5]
 [ 78815  87257 136412 183844 275766]]


In [10]:
print("feature " + str(data_np.shape))


feature (762094, 43)


In [11]:
feature = data_np[:, :-1]
# target = data_np[:, -1].reshape(data_np.shape[0], 1)
target = data_np[:, -1]


In [12]:
# for x in range(10):
#     print(target[x])

In [13]:
target -= 1
# for x in range(10):
#     print(target[x])

In [14]:
X_trainAndVal, X_test, y_trainAndVal, y_test = train_test_split(feature, target, test_size = 0.2, random_state = 0)

In [15]:
X_train, X_val, y_train, y_val = train_test_split(X_trainAndVal, y_trainAndVal, test_size = 0.2, random_state = 0)

In [16]:
print((X_train ).shape)
print(( y_train).shape)


(487740, 42)
(487740,)


In [17]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)
X_test = sc.transform(X_test)

In [18]:
batch_size = 1000
epochs = 50
input_dim = first_feature_count + last_feature_count - 1
output_dim = 5
lr_rate = 0.001


In [19]:
class datasetLoad(Dataset):
    def __init__(self, features,labels):
        self.features = features
        self.labels = labels
    def __len__(self):
        return len(self.features)
    def __getitem__(self, index):
        return self.features[index], self.labels[index]
    


In [20]:
X_train = datasetLoad(X_train, y_train)
X_val = datasetLoad(X_val, y_val)
X_test = datasetLoad(X_test, y_test)


In [21]:
train_loader = torch.utils.data.DataLoader(dataset = X_train, batch_size = batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset = X_val, batch_size = batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset = X_test, batch_size = batch_size, shuffle=True)

In [22]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs
    
    

In [23]:
model = LogisticRegression(input_dim, output_dim)

In [24]:
criterion = torch.nn.CrossEntropyLoss()


In [25]:
# sgd ya da adam kullanilabilir
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [26]:
print(len(X_train))

487740


In [ ]:
import datetime
a = datetime.datetime.now().replace(microsecond=0)
for epoch in range(epochs):
    for i, (features, labels) in enumerate(train_loader):
        features = Variable(features)
        labels = Variable(labels)

        optimizer.zero_grad()
        outputs = model(features.float())
        loss = criterion(outputs.float(), labels)
        loss.backward()
        optimizer.step()

        
        if i % (len(X_train)// (4 * batch_size))  == 0:
            # calculate Accuracy
            val_correct = 0
            val_total = 0
            for val_features, val_labels in val_loader:
                val_features = Variable(val_features)
                val_outputs = model(val_features.float())
                _, val_predicted = torch.max(val_outputs.data, 1)
                val_total+= val_labels.size(0)
                # for gpu, bring the predicted and labels back to cpu fro python operations to work
                val_correct+= (val_predicted == val_labels).sum()
            val_accuracy = 100 * val_correct / val_total
            if(epoch % 5 == 0 and i == 0):
                print("val Epoch: {}. Iteration: {}. Loss: {}. Accuracy: {}.".format(epoch, i, loss.item(), val_accuracy))
             


val Epoch: 0. Iteration: 0. Loss: 1.6722856760025024. Accuracy: 24.
val Epoch: 5. Iteration: 0. Loss: 1.4745162725448608. Accuracy: 40.
val Epoch: 10. Iteration: 0. Loss: 1.3596712350845337. Accuracy: 42.
val Epoch: 15. Iteration: 0. Loss: 1.3268316984176636. Accuracy: 43.
val Epoch: 20. Iteration: 0. Loss: 1.3236610889434814. Accuracy: 43.


In [ ]:
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)